In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import norm

In [ ]:
df=pd.read_csv('../input/wine-quality/winequalityN.csv')

GETTING TO KNOW OUR DATA

In [ ]:
print(df.shape)
print(df.columns)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

ANALYSING QUALITY FUNCTION

In [ ]:
df['quality'].describe()

In [ ]:
dfv=df['quality'].value_counts()
print(dfv)

In [ ]:
sns.histplot(df['quality'], binwidth=1,color='blue');

In [ ]:
sns.distplot(df['quality'],bins=7);

In [ ]:
print(df['quality'].skew())
print(df['quality'].kurt())

Seeing the coorelation of variables with 'Quality' variable


In [ ]:
plt.figure(figsize=(16,6))
sns.heatmap(df.corr(),annot=True);

In [ ]:
#coorelation with categorical variables
var = 'type'
data = pd.concat([df['quality'], df[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="quality", data=data)

In [ ]:
for a in range(len(df.corr().columns)):
    for b in range(a):
        if abs(df.corr().iloc[a,b]) >0.7:
            name = df.corr().columns[a]
            print(name)

In [ ]:
new_df=df.drop('total sulfur dioxide',axis=1)

HANDLING MISSING VALUES

In [ ]:
new_df.isnull().sum()

In [ ]:
new_df.update(new_df.fillna(new_df.mean()))

In [ ]:
#missing data
total = new_df.isnull().sum().sort_values(ascending=False)
percent = (new_df.isnull().sum()/new_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
# catogerical vars 
next_df = pd.get_dummies(new_df,drop_first=True)
# display new dataframe
next_df

In [ ]:
next_df1=next_df

In [ ]:
next_df1["best quality"] = [ 1 if x>=7 else 0 for x in df.quality] 
print(next_df1)

In [ ]:
next_df1.columns

In [ ]:
features=(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'density', 'pH', 'sulphates',
       'alcohol','type_white', 'best quality'])

In [ ]:
X=next_df[features]
y=next_df['quality']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

USING MINMAXSCALER

In [ ]:
#importing module
from sklearn.preprocessing import MinMaxScaler
# creating normalization object 
norm = MinMaxScaler()
# fit data
norm_fit = norm.fit(X_train)
new_Xtrain = norm_fit.transform(X_train)
new_Xtest = norm_fit.transform(X_test)
# display values
print(new_Xtrain)

USING RANDOM CLASSIFIER WITH MINMAXSCALER

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error

rfc=RandomForestClassifier(max_depth=30,n_estimators=200,random_state=0)
fit_rfc=rfc.fit(new_Xtrain, y_train)
rfc_score_train=rfc.score(new_Xtrain, y_train)
rfc_score_test=rfc.score(new_Xtest, y_test)
print(rfc_score_train)
print(rfc_score_test)